In [1]:
import pandas as pd
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA 
from pandas.tools.plotting import autocorrelation_plot
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
#Install the missing packages by: python -m pip install <package name> or pip install <package name>
class CSPredict:
    def __init__(self,path):
        self.path=path
        
    def parser(self,x):
        return x
    def start(self):
        # read and parse data 
        print("#############################################################")
        print("###################CITY SCORE PREDICTION #############")
        print("#############################################################")
        print("-------------------------------------------------------------")
        city_score = read_csv(self.path, header=None,parse_dates=[2], squeeze=True, date_parser=self.parser)
        city_score = city_score.iloc[:, [2, 1]]
        city_score.columns = ['Date', 'Score']

        # set index as datetime to support plotting and Arima Model
        city_score.set_index('Date', inplace=True)
        # city_score = city_score.asfreq('T')

        print(city_score.head())

        city_score.plot(title='City Score Prediction')
        pyplot.tight_layout()
        pyplot.show()
        print("Printing the statistics")
        print("-----------------------------------------")
        self.stats(city_score)
    def stats(self,city_score):
        model = ARIMA(city_score, order=(5,1,0))
        model_fit = model.fit(disp=0)
        print(model_fit.summary())

        # plot residual errors
        residuals = pd.DataFrame(model_fit.resid)
        residuals.plot()
        pyplot.show()
        residuals.plot(kind='kde')
        pyplot.show()
        print(residuals.describe())
        print("-----------------------------------------")
        print("Please wait...... Training the testcases is in progress...")
        print("-----------------------------------------")
        self.autocorelaton(city_score)
    def autocorelaton(self,city_score):
        autocorrelation_plot(city_score)
        pyplot.show()
        #Will take more than 30 minutes to 1 hour to run as it will train the data.
        X = city_score.values
        size = int(len(X) * 0.66)
        train, test = X[0:size], X[size:len(X)]
        history = [x for x in train]
        predictions = list()
        print('Total Test cases: {}'.format(len(test)))
        for t in tqdm(range(len(test))):
            model = ARIMA(history, order=(5,1,0))
            model_fit = model.fit(disp=0)
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            obs = test[t]
            history.append(obs)
        self.finalize(test,predictions)
    def finalize(self,test,predictions):
        error = mean_squared_error(test, predictions)
        print('Test MSE: %.3f' % error)
        # plot
        pyplot.plot(test)
        pyplot.plot(predictions, color='red')
        pyplot.show()
        print("------------------------------------------")
        print("END")

CSPredict('cityscore.csv').start();


#############################################################
###################CITY SCORE PREDICTION #############
#############################################################
-------------------------------------------------------------


C:\Users\Sunita\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


               Score
Date                
2016-06-28  0.916239
2016-04-13  0.964654
2016-04-15  0.945004
2016-07-22  0.906451
2017-10-02  0.913069


Printing the statistics
-----------------------------------------


C:\Users\Sunita\Anaconda3\lib\site-packages\statsmodels\tsa\kalmanf\kalmanfilter.py:646: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if issubdtype(paramsdtype, float):
C:\Users\Sunita\Anaconda3\lib\site-packages\statsmodels\tsa\kalmanf\kalmanfilter.py:650: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  elif issubdtype(paramsdtype, complex):


...

C:\Users\Sunita\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning: 'pandas.tools.plotting.autocorrelation_plot' is deprecated, import 'pandas.plotting.autocorrelation_plot' instead.


Total Test cases: 217


100%|██████████| 217/217 [00:54<00:00,  3.95it/s]


Test MSE: 0.001


------------------------------------------
END
